In [1]:
import pandas as pd
from piex import explorer

from mit_d3m.db import get_db

Using TensorFlow backend.


In [146]:
db = get_db(config='../mongodb_config.json')

In [147]:
piex = explorer.MongoPipelineExplorer(db)

In [148]:
import numpy as np

def normalize(metric_type, min_value=None, max_value=None):
    def f(raw):
        if metric_type == 'zero_one_score':
            return raw
        elif metric_type == 'zero_one_cost':
            return 1 - raw
        elif metric_type == 'ranged_score':
            return (raw - min_value) / (max_value - min_value)
        elif metric_type == 'real_score':
            return 1 / (1 + np.exp(-raw))
        elif metric_type == 'real_cost':
            return 1 - (1 / (1 + np.exp(-raw)))
        elif metric_type == 'zero_inf_score':
            return 1 / (1 + np.exp(-np.log10(raw)))
        elif metric_type == 'zero_inf_cost':
            return 1 - 1 / (1 + np.exp(-np.log10(raw)))
        else:
            raise ValueError('Unknown metric type')
    
    return f

In [189]:
METRIC_TYPES = {
    'f1': 'zero_one_score',
    'f1Macro': 'zero_one_score',
    'accuracy': 'zero_one_score',
    'normalizedMutualInformation': 'zero_one_score',
    'meanSquaredError': 'zero_inf_cost',
    'rootMeanSquaredError': 'zero_inf_cost',
}

In [150]:
def normalize_score(metric_column, score_column):
    def f(row):
        return normalize(METRIC_TYPES[row.metric])(row.cv_score)
    
    return f

In [152]:
def get_best(group):
    return group.sort_values('t-score', ascending=False).iloc[0]

def best_score_by_dataset(df):
    return df.groupby('dataset').apply(get_best)['t-score']

In [229]:
def get_wins(dfs, conditions):
    wins = []
    (name_a, condition_a), (name_b, condition_b) = list(conditions.items())
    for name, df in dfs.items():
        df['t-score'] = df.apply(normalize_score('metric', 'cv_score'), axis=1)

        best_scores = pd.DataFrame({
            name_a: best_score_by_dataset(df[condition_a(df)]),
            name_b: best_score_by_dataset(df[condition_b(df)])
        }).fillna(0)

        wins.append(pd.Series({
                name_a: (best_scores[name_a] > best_scores[name_b]).sum(),
                name_b: (best_scores[name_a] < best_scores[name_b]).sum(),
                'TIE': (best_scores[name_a] == best_scores[name_b]).sum()
            }, name=name))
    
    return pd.DataFrame(wins).T

# XGB vs RF

## Single table Classification + Regression

In [156]:
st_classification = piex.get_test_results(data_modality='single_table', task_type='classification')
st_regression = piex.get_test_results(data_modality='single_table', task_type='regression')

In [235]:
def is_xgb(df):
    return df.pipeline.str.contains('xgb').fillna(False)

def is_rf(df):
    return df.pipeline.str.contains('random_forest').fillna(False)

In [243]:
dfs = {
    'single_table/classification': st_classification,
    'single_table/regression': st_regression,
}
xgb_vs_rf = {
    'XGB': is_xgb,
    'RF': is_rf,
}

In [239]:
get_wins(dfs, xgb_vs_rf)

,single_table/classification,single_table/regression
XGB,169,61
RF,51,25
TIE,14,0


## Graph

In [234]:
graph = piex.get_test_results(
    data_modality='graph',
    task_type={
        '$in': [
            'link_prediction',
            'vertex_nomination',
            'graph_matching'
        ]
    }
).dropna(subset=['cv_score'])

In [249]:
def is_xgb(df):
    return df.pipeline.str.contains('default').fillna(True)

def is_rf(df):
    return df.pipeline.str.contains('random_forest').fillna(False)

xgb_vs_rf = {
    'XGB': is_xgb,
    'RF': is_rf,
}

In [251]:
get_wins({'graph': graph}, xgb_vs_rf)

,graph
XGB,4
RF,5
TIE,2


# DFS vs No DFS

In [182]:
def is_dfs(df):
    return df.pipeline.str.startswith('dfs').fillna(False)

def is_no_dfs(df):
    return ~df.pipeline.str.startswith('dfs').fillna(True)

In [183]:
dfs_vs_no_dfs = {
    'DFS': is_dfs,
    'No DFS': is_no_dfs
}

In [184]:
get_wins(dfs, dfs_vs_no_dfs)

,single_table/classification,single_table/regression
DFS,47,16
No DFS,153,67
TIE,34,3
